In [ ]:
!pip install datasets
from datasets import load_dataset
from transformers import GPTNeoXForCausalLM, AutoTokenizer
import torch
from tqdm import tqdm
import pandas as pd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 7.6 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [ ]:
small_model_param_string='70m'
large_model_param_string='160m'
# model_param_string='410m'
# model_param_string='1b'
# model_param_string='1.4b'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# set up small model
model_s = GPTNeoXForCausalLM.from_pretrained(
  f"EleutherAI/pythia-{small_model_param_string}-deduped",
  revision="step3000",
  cache_dir=f"./pythia-{small_model_param_string}-deduped/step3000",
)

tokenizer_s = AutoTokenizer.from_pretrained(
  f"EleutherAI/pythia-{small_model_param_string}-deduped",
  revision="step3000",
  cache_dir=f"./pythia-{small_model_param_string}-deduped/step3000",
)

prompt = 'Independence Day, known colloquially as the Fourth of July,'
inputs = tokenizer_s(prompt, return_tensors="pt")
tokens = model_s.generate(**inputs)
tokenizer_s.decode(tokens[0])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/567 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/166M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/396 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


'Independence Day, known colloquially as the Fourth of July, the Supreme Court of the United States'

In [ ]:
# set up large model
model_L = GPTNeoXForCausalLM.from_pretrained(
  f"EleutherAI/pythia-{large_model_param_string}-deduped",
  revision="step3000",
  cache_dir=f"./pythia-{large_model_param_string}-deduped/step3000",
)

tokenizer_L = AutoTokenizer.from_pretrained(
  f"EleutherAI/pythia-{large_model_param_string}-deduped",
  revision="step3000",
  cache_dir=f"./pythia-{large_model_param_string}-deduped/step3000",
)

prompt = 'Independence Day, known colloquially as the Fourth of July,'
inputs = tokenizer_L(prompt, return_tensors="pt")
tokens = model_L.generate(**inputs)
tokenizer_L.decode(tokens[0])

config.json:   0%|          | 0.00/569 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/375M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/396 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


'Independence Day, known colloquially as the Fourth of July, is a day of celebration, and'

In [ ]:
'''
Download the CSV file and upload to your Colab workspace
'''

# Read the CSV file into a DataFrame
df = pd.read_csv('top_n_data_all.csv')

# Convert CSV outputs to list of stings
seqs = [(row['output']) for _, row in df.iterrows()]

In [ ]:
# Function to calculate perplexity
def calculate_perplexity(nlls):
    total_loss = sum(nlls)
    avg_loss = total_loss / len(nlls)
    perplexity = torch.exp(avg_loss)
    return perplexity

# store perplexities of small and large models
ppl_s = []
ppl_L = []
for seq in tqdm(seqs, desc="Processing sequences"):
    encodings_s = tokenizer_s(seq, return_tensors="pt")
    encodings_L = tokenizer_L(seq, return_tensors="pt")
    max_length = model_s.config.max_position_embeddings
    stride = 512
    seq_len = encodings_s.input_ids.size(1)

    nlls_s = []
    nlls_L = []
    prev_end_loc = 0
    for begin_loc in range(0, seq_len, stride):
        end_loc = min(begin_loc + max_length, seq_len)
        trg_len = end_loc - prev_end_loc  # may be different from stride on last loop
        input_ids_s = encodings_s.input_ids[:, begin_loc:end_loc].to(device)
        target_ids_s = input_ids_s.clone()
        target_ids_s[:, :-trg_len] = -100
        input_ids_L = encodings_L.input_ids[:, begin_loc:end_loc].to(device)
        target_ids_L = input_ids_L.clone()
        target_ids_L[:, :-trg_len] = -100

        with torch.no_grad():
            outputs_s = model_s(input_ids_s, labels=target_ids_s)
            outputs_L = model_L(input_ids_L, labels=target_ids_L)

            # loss is calculated using CrossEntropyLoss which averages over valid labels
            # N.B. the model only calculates loss over trg_len - 1 labels, because it internally shifts the labels
            # to the left by 1.
            neg_log_likelihood_s = outputs_s.loss
            neg_log_likelihood_L = outputs_L.loss

        nlls_s.append(neg_log_likelihood_s)
        nlls_L.append(neg_log_likelihood_L)

        prev_end_loc = end_loc
        if end_loc == seq_len:
            break

    perplexity = calculate_perplexity(nlls_s)
    ppl_s.append(perplexity)
    perplexity = calculate_perplexity(nlls_L)
    ppl_L.append(perplexity)

print(ppl_s)
print(ppl_L)


Processing sequences: 100%|██████████| 2000/2000 [30:56<00:00,  1.08it/s]


[tensor(21.9335), tensor(13.5053), tensor(13.8157), tensor(22.4356), tensor(17.7937), tensor(17.5451), tensor(24.8752), tensor(9.6335), tensor(7.9203), tensor(24.8191), tensor(26.9920), tensor(24.4554), tensor(21.3393), tensor(13.6263), tensor(17.3550), tensor(29.0606), tensor(21.5657), tensor(18.7864), tensor(17.2466), tensor(18.7003), tensor(25.3955), tensor(39.7694), tensor(19.4945), tensor(21.3684), tensor(20.1139), tensor(25.5967), tensor(37.8209), tensor(22.9590), tensor(26.0179), tensor(3.8479), tensor(20.7459), tensor(21.9427), tensor(28.5871), tensor(29.6048), tensor(8.1596), tensor(28.2044), tensor(16.1223), tensor(23.0577), tensor(26.2433), tensor(20.8756), tensor(33.5557), tensor(17.9701), tensor(23.3895), tensor(20.4204), tensor(29.5900), tensor(14.4091), tensor(50.8596), tensor(24.1951), tensor(22.7180), tensor(33.7275), tensor(24.2140), tensor(13.8395), tensor(13.9873), tensor(23.6246), tensor(30.9974), tensor(22.7131), tensor(21.8162), tensor(26.6226), tensor(9.7134), t

In [ ]:
import numpy as np
# ppl_small = [tensor.cpu().numpy() for tensor in ppl_s]
# ppl_Large = [tensor.cpu().numpy() for tensor in ppl_L]

# # Save the lists of NumPy arrays separately
# np.save('ppl_s.npy', ppl_small)
# np.save('ppl_L.npy', ppl_Large)

# Load the first list of perplexities
ppl_small = np.load('ppl_s.npy', allow_pickle=True)
ppl_Large = np.load('ppl_L.npy', allow_pickle=True)

In [ ]:
ratio = np.log(ppl_Large)/np.log(ppl_small)

In [ ]:
import zlib

# Calculate zlib entropy for each sequence
zlib_entropies = []
for seq in seqs:
    compressed_data = zlib.compress(seq.encode())
    entropy = len(compressed_data) / len(seq)
    zlib_entropies.append(entropy)

ratio = np.log(ppl_Large)/zlib_entropies
# Print or use zlib_entropies as needed
print(zlib_entropies)

[0.580168776371308, 0.5652173913043478, 0.6006493506493507, 0.5670103092783505, 0.5128205128205128, 0.5717821782178217, 0.5642701525054467, 0.5326876513317191, 0.43213296398891965, 0.5328467153284672, 0.7612612612612613, 0.7564935064935064, 0.7576923076923077, 0.6736401673640168, 0.757847533632287, 0.7333333333333333, 0.628808864265928, 0.7328990228013029, 0.5601965601965602, 0.717391304347826, 0.520891364902507, 0.6072507552870091, 0.5995145631067961, 0.6276276276276276, 0.6309859154929578, 0.6633986928104575, 0.6578073089700996, 0.6679389312977099, 0.6288343558282209, 0.23964497041420119, 0.6010362694300518, 0.585, 0.5885286783042394, 0.6229508196721312, 0.47003154574132494, 0.5851851851851851, 0.6302521008403361, 0.6894586894586895, 0.5337423312883436, 0.6487341772151899, 0.5671296296296297, 0.5698924731182796, 0.610126582278481, 0.6098081023454158, 0.625, 0.6452599388379205, 0.5812356979405034, 0.5835351089588378, 0.573394495412844, 0.5561224489795918, 0.6191709844559585, 0.6181318

In [ ]:
# Convert tensors to numerical values (e.g., floats)
# ppl_values = [tensor.item() for tensor in ppl_L]

# Combine sequences and perplexities into tuples
combined = list(zip(seqs, ratio))

# Sort the combined list based on perplexity values (low to high)
sorted_combined = sorted(combined, key=lambda x: x[1])

# Extract sorted sequences
sorted_seqs = [item[0] for item in sorted_combined]

In [ ]:
import pandas as pd
df = pd.DataFrame(sorted_combined, columns=['generation', 'zlib'])
df.to_csv('zlib_sorted.csv')